In [ ]:
import pandas as pd
import glob
from datetime
import matplotlib.pyplot as plt

out_dir = "../data" #output directory
# read data from all files
result = pd.concat(
    [pd.read_csv(file, encoding="utf-8", parse_dates=[1,2,5], dayfirst=True) for file in glob.glob(out_dir + "/*.csv")],
    ignore_index=True)

In [ ]:
#     'ostapbender', 'nvaluev', 'karpovvyacheslav', 'naguchevroman'
##    'chaplygin', 'kozhevnikov', 'netsenko', 'zhukov', 'falcao1984', 'andronov', 'teplofevralya', 'nobel'